In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='ffe2b28ed89637889c04405e5d7d2d98be3df5b6', drake_version='0.27.0', drake_build='release')

In [2]:
# Do general python imports
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Drake Imports

from pydrake.all import MultibodyPlant, Parser
from pydrake.all import Linearize, LinearQuadraticRegulator, IsControllable, ControllabilityMatrix

In [3]:
box_urdf = """
<?xml version="1.0"?>
<robot name="box" >
    <link name="box_link">
        <inertial>
            <origin xyz="0 0 0" rpy="0 0 0"/>
            <mass value="1"/>
            <inertia ixx="100"  ixy="0"  ixz="0" iyy="100" iyz="0" izz="100" />
        </inertial>
        <visual>
            <origin xyz="0 0 0" rpy="0 0 0" />
            <geometry>
                <box size="1. 1. 2." />
            </geometry>
            <material name="Cyan">
                <color rgba="0 1.0 1.0 1.0"/>
            </material>
        </visual>
    </link>
</robot>
"""

In [4]:
# Setup Drake Multibody Plant

SC_plant = MultibodyPlant(time_step=0.0)
# scene_graph = SceneGraph()
# SC_plant.RegisterAsSourceForSceneGraph(scene_graph)
SC_instance = Parser(SC_plant).AddModelFromString(box_urdf, "urdf")
# Remove gravity
plantGravityField = SC_plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
SC_plant.Finalize()

SC_context = SC_plant.CreateDefaultContext()

# Use Generalized Forces for Linearization
generalized_forces_port = SC_plant.get_applied_generalized_force_input_port()
generalized_forces_port.FixValue(SC_context, np.zeros((generalized_forces_port.size(), 1)))

In [5]:
SC_plant_linear = Linearize(SC_plant, SC_context,
                            input_port_index=SC_plant.get_applied_generalized_force_input_port().get_index(),
                            output_port_index=SC_plant.get_state_output_port().get_index())

In [6]:
AMatrix = SC_plant_linear.A()
BMatrix = SC_plant_linear.B()
CMatrix = SC_plant_linear.C()
DMatrix = SC_plant_linear.D()

In [7]:
IsControllable(SC_plant_linear)

False

In [8]:
ctrlb = ControllabilityMatrix(SC_plant_linear)

In [9]:
np.linalg.matrix_rank(ctrlb)

12

In [10]:
# LinearQuadraticRegulator(AMatrix, BMatrix, np.ones(13), np.ones((6, 1)))

In [18]:
import control

In [26]:
Q = np.ones(13)
R = np.ones(6)

Q = np.diag(Q)
R = np.diag(R)
print(Q)
print(R)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [27]:
control.lqr(AMatrix, BMatrix, Q, R)

SlycotArithmeticError: 
The Hamiltonian or symplectic matrix H has less
than n stable eigenvalues;